In [ ]:
# LIBRARIES

import torch
torch.cuda.empty_cache() # clean cache memory
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from torch.utils.data import ConcatDataset, TensorDataset, random_split, Dataset
import torchvision.datasets as datasets
import matplotlib.pyplot as plt
import random
import numpy as np
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR

# To be able to apply transforms to a subset ... 
class MyDataset(Dataset):
    def __init__(self, subset, transform=None):
        self.subset = subset
        self.transform = transform
        
    def __getitem__(self, index):
        x, y = self.subset[index]
        if self.transform:
            x = self.transform(x)
        return x, y
        
    def __len__(self):
        return len(self.subset)
    
    def get_labels(self):
        return self.subset.targets


# Make sure I can run on my device's GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

torch.manual_seed(0)

In [ ]:
# DOWNLOAD THE DATASET INTO FOLDERS (URLs IN EXCEL - FOLDERS)

import requests
import os
import pandas as pd

# Read the excel file
df = pd.read_excel('incidencias_limpias_4.xlsx')

# Create one folder per class
clases = df['Clase'].unique()
for clase in clases:
    os.makedirs(clase, exist_ok=True)

# Go through the dataframe and download the URLs
for i, row in df.iterrows():
    url = row['URL']
    clase = row['Clase']
    nombre = row['Nombre']
    extension = url.split('.')[-1]
    nombre_archivo = f"{nombre}.{extension}"
    ruta_carpeta = clase
    ruta_archivo = os.path.join(ruta_carpeta, nombre_archivo)
    
    # Download the image and save in its folder
    response = requests.get(url)
    with open(ruta_archivo, 'wb') as f:
        f.write(response.content)

In [ ]:
# LOADING AND DATA AUGMENTATION

# Link to the path folder that contains the images
data_path = "dataset"

# Image transformation
transform = transforms.Compose([
    transforms.CenterCrop((500)),
    transforms.Resize((260, 260)),
    transforms.ToTensor(),
    transforms.Normalize([0.4887, 0.4764, 0.4461],[0.2396, 0.2302, 0.2442])
])

# Create the main dataset from the folder containing all images and classes
dataset = datasets.ImageFolder(root=data_path, transform=transform)

# Divide the dataset into training and validation
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

# Define data augmentation transforms
augmentation_transforms = transforms.Compose([
    transforms.RandomRotation(15),
    transforms.RandomHorizontalFlip(100),
])

# Create augmented dataset
augmented_train_dataset = MyDataset(train_dataset, augmentation_transforms) # applies transformation to my subset

# Concatenate datasets
final_train_dataset = ConcatDataset([train_dataset, augmented_train_dataset])

In [ ]:
# PRINT A RANDOM IMAGE 

# To make sure the datasets are right
# Select a random image from the dataset
image, label = random.choice(dataset) # use final_train_dataset to plot an image from the whole dataset (includes augmented)

# plot an image via matplotlib
plt.imshow(image.permute(1, 2, 0)) # traspose the image for it to be shown correctly
plt.show()

# show image class
print(dataset.classes[label])

In [ ]:
# TRAINING AND VALIDATION

# Load pre-trained MobileNetV2 model. Train the las 3 conv. layers and the last F.C.
model = models.mobilenet_v2(pretrained=True)

# Define data loaders
batch_size = 32
train_loader = torch.utils.data.DataLoader(final_train_dataset, batch_size=batch_size, shuffle=True) #train_dataset or final_train_dataset para aplicar data augmentation
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=True)

# Freeze all layers except the last two
for param in model.parameters():
    param.requires_grad = False
for param in model.features[-1].parameters():
    param.requires_grad = True
for param in model.features[-2].parameters():
    param.requires_grad = True
for param in model.features[-3].parameters():
    param.requires_grad = True
#for param in model.features[-4].parameters():
#    param.requires_grad = True
for param in model.classifier.parameters():
    param.requires_grad = True
    
# Add dropout layer
model.features.add_module('dropout', nn.Dropout(p=0.1))

# Modify the last layer for our dataset
num_classes = len(dataset.classes)
in_features = model.classifier[1].in_features
model.classifier[1] = nn.Linear(in_features, num_classes)

# Move model to device
model = model.to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001,betas=(0.9,0.999),weight_decay=0.001)

# defino el reductor de lr
step_size = 4   # every "step_size" n of epochs
gamma = 0.5     # lr multiplied by "gamma"

scheduler = StepLR(optimizer, step_size=step_size, gamma=gamma)

# Train the model
num_epochs = 25
train_losses, train_accs, val_losses, val_accs = [], [], [], [] # define them, will be plotted
for epoch in range(num_epochs):
    # Train the model for one epoch
    model.train()
    train_loss = 0
    train_correct = 0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * inputs.size(0)
        train_correct += torch.sum(torch.argmax(outputs, dim=1) == labels)

    # Evaluate the model on the validation set
    model.eval()
    val_loss = 0
    val_correct = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * inputs.size(0)
            val_correct += torch.sum(torch.argmax(outputs, dim=1) == labels)

    # Calculate metrics
    train_loss /= len(train_loader.dataset)
    train_acc = train_correct.float() / len(train_loader.dataset)
    val_loss /= len(val_loader.dataset)
    val_acc = val_correct.float() / len(val_loader.dataset)
    
    # Defined to plot loss curves
    val_losses.append(val_loss)
    val_accs.append(val_acc)
    train_losses.append(train_loss)
    train_accs.append(train_acc)

    # Print progress
    print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")
    scheduler.step()
    
    
# Plot the loss curves
plt.plot(train_losses, label="Train")
plt.plot(val_losses, label="Validation")
plt.title("Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()
plt.ylim(0,max(train_losses)*1.1)
plt.savefig('loss_curves.eps', format='eps')
plt.show()

# Plot the accuracy curves
cpu_train_accs = []
for tensor in train_accs:
    cpu_train_accs.append(tensor.cpu())
plt.plot(cpu_train_accs, label="Train")

cpu_val_accs = []
for tensor in val_accs:
    cpu_val_accs.append(tensor.cpu())
plt.plot(cpu_val_accs, label="Validation")

plt.title("Accuracy")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend()
plt.ylim(0,max(cpu_train_accs)*1.1)
plt.savefig('acc_curves.eps', format='eps')
plt.show()

In [ ]:
# CONFUSION MATRIX

from sklearn.metrics import confusion_matrix


# Get class names from dataset
class_names = val_dataset.dataset.classes

# Evaluate the model on the validation set and calculate the confusion matrix
model.eval()
y_true, y_pred = [], []
with torch.no_grad():
    for images, labels in val_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        y_true += labels.cpu().numpy().tolist()
        y_pred += predicted.cpu().numpy().tolist()

confusion = confusion_matrix(y_true, y_pred, labels=range(len(class_names)))

def plot_confusion_matrix(cm, classes, normalize=True, title='Matriz de confusión', cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    fig, ax = plt.subplots(figsize=(8, 8))
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           xticklabels=classes, yticklabels=classes,
           title=title,
           ylabel='True label',
           xlabel='Predicted label')

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=90, ha="right",
             rotation_mode="anchor")
    
    # Adjust spacing between axes and figure borders
    fig.subplots_adjust(left=0.15, bottom=0.15, right=0.95, top=0.95)
    
    return ax

# Plot the confusion matrix with class labels
plot_confusion_matrix(confusion, classes=class_names, title='Matriz de confusión')
plt.savefig('matx.eps', format='eps')
plt.show()

In [ ]:
# ACCURACY DISTRIBUTION

accuracy = np.diag(confusion) / np.sum(confusion, axis=1) * 100
aciertos=np.diag(confusion)

# Sort the labels and accuracy values in descending order
sorted_indices = np.argsort(accuracy)
sorted_labels = np.array(class_names)[sorted_indices]
sorted_accuracy = accuracy[sorted_indices]
sorted_aciertos = np.diag(confusion)[sorted_indices]

# Create the graph 
fig, ax = plt.subplots(figsize=(8, 5))

# Modify sorted_labels with sorted_accuracy
ax.barh(sorted_labels, sorted_accuracy, color='steelblue') 
ax.set_xlabel('% de Acierto')  
ax.tick_params(axis='x')  # Use axis='x' instead of axis='y'
plt.setp(ax.get_yticklabels(), rotation=0, ha="right", rotation_mode="anchor")

# Show number of right answers next to each bar
for i, (acc, aciertos) in enumerate(zip(sorted_accuracy, sorted_aciertos)):
    ax.text(acc, i, f'{aciertos}', ha='left', va='center')

# Other configurations of te graph
ax.set_ylabel('Etiqueta')  

# Plot and save the graph
plt.show()
fig.savefig('pareto.eps', format='eps', dpi=300, bbox_inches='tight')